In [3]:
import matplotlib.pyplot as plt

In [ ]:
# !pip install matplotlib

  Using cached matplotlib-3.9.4-cp39-cp39-win_amd64.whl (7.8 MB)
  Using cached kiwisolver-1.4.7-cp39-cp39-win_amd64.whl (55 kB)
  Using cached pillow-11.3.0-cp39-cp39-win_amd64.whl (7.0 MB)
     ---------------------------------------- 1.5/1.5 MB 10.8 MB/s eta 0:00:00
  Using cached importlib_resources-6.5.2-py3-none-any.whl (37 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl (111 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached contourpy-1.3.0-cp39-cp39-win_amd64.whl (211 kB)


You should consider upgrading via the 'B:\Projects\the-world-trade\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
# !pip install pandas numpy seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)


You should consider upgrading via the 'B:\Projects\the-world-trade\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
df_raw = pd.read_csv("world_trade_synth_fast.csv")

In [7]:
df_raw.head(10)

,year,reporter_iso3,partner_iso3,product_code,distance_km,fta_active,adval_tariff_pct,reporter_gdp_bln,partner_gdp_bln,reporter_pop_m,partner_pop_m,reporter_cpi,partner_cpi,export_value_usd_mln,quantity_tonnes,unit_price_usd_per_tonne
0,1975,USA,DZA,AGRI,5273.4,0,9.23,180.484,443.922,48.765,10.598,51.95,53.25,1.703,58.330,29191.463644
1,1975,USA,DZA,ENER,5273.4,0,14.81,180.484,443.922,48.765,10.598,51.95,53.25,2.213,75.814,29191.463644
2,1975,USA,DZA,META,5273.4,0,9.57,180.484,443.922,48.765,10.598,51.95,53.25,0.883,30.262,29191.463644
3,1975,USA,DZA,CHEM,5273.4,0,9.15,180.484,443.922,48.765,10.598,51.95,53.25,4.917,168.447,29191.463644
4,1975,USA,DZA,MACH,5273.4,0,3.99,180.484,443.922,48.765,10.598,51.95,53.25,15.236,521.919,29191.463644
5,1975,USA,MAR,AGRI,6073.4,0,9.23,180.484,145.029,48.765,41.568,51.95,53.44,0.511,17.498,29191.463644
6,1975,USA,MAR,ENER,6073.4,0,14.81,180.484,145.029,48.765,41.568,51.95,53.44,0.691,23.684,29191.463644
7,1975,USA,MAR,META,6073.4,0,9.57,180.484,145.029,48.765,41.568,51.95,53.44,0.694,23.759,29191.463644
8,1975,USA,MAR,CHEM,6073.4,0,9.15,180.484,145.029,48.765,41.568,51.95,53.44,1.767,60.530,29191.463644
9,1975,USA,MAR,MACH,6073.4,0,3.99,180.484,145.029,48.765,41.568,51.95,53.44,0.830,28.439,29191.463644
